In [3]:
import numpy as np
import sklearn.datasets

In [201]:
"""
- decision trees are greedy: they will take the predictor that minimizes the chosen loss function at each level.
- To decide, we split the tree into leaves based on the label values of a predictor. The more "pure" each leaf is,
  the better that split is. So we choose the split that makes the best leaves.
- Gini Impurity: 1 - p(being in one group)^2 - p(probability of being in the other group)^2
- different for numeric data: need to sort obs from lowest to highest value. Then split sequentially starting with 
  there being two leaves of size 1 and n-1, and finishing with leaves of size n-1 and 1. Calculate the weighted 
  average impurity of each split, pick the split that gives the lowest average impurity.
- continue splitting tree until we reach some depth k or we reach a pure node

process for decision tree: 
    1. for each variable, calculate the gini impurity of using it to split the target into groups
        a. For categorical variables, do this by splitting into groups based on category labels and
           calculating the gini impurity for each new node. Then average that impurity for a score.
        b. For numeric variables, sort the values and create an expanding window of splits into a
           less than leaf and a greater than leaf. Choose the split pair that minimized the gini impurity.
    2. Recursively do this for each new node until you reach completely pure nodes OR you reach an 
       arbitrary depth k. 
    3. To predict new values, just run it through the prebuilt decision tree.
"""


class dTree:
    def __init__(self, k=0):
        self.l_child = None
        self.r_child = None
        self.k = k
        self.split_col = None
        self.split_val = None
        self.is_leaf = False
        self.y_vals = []
        
    def find_best_split(self, data, target):
        min_gini = 9999999
        best_col = None
        best_split = None

        for col_idx in range(len(data[0])):
            sorted_x = data[data[:, col_idx].argsort()]
            sorted_y = target[data[:, col_idx].argsort()]

            for row_idx in range(len(data) - 1):
                lesser_half = sorted_y[sorted_x[:, col_idx] <= sorted_x[row_idx][col_idx]]
                greater_half = sorted_y[sorted_x[:, col_idx] > sorted_x[row_idx][col_idx]]
                avg_gini = (gini(lesser_half) + gini(greater_half)) / 2
                if avg_gini < min_gini:
                    best_col = col_idx
                    best_split = sorted_x[row_idx][col_idx]
                    min_gini = avg_gini
                if min_gini == 0:
                    break
            if min_gini == 0:
                break
                
            
        return best_col, best_split, min_gini
    
    def fit(self, X, y):
        split_col, split_val, min_gini = self.find_best_split(X, y)
        self.split_col = split_col
        self.split_val = split_val

        lesser_criteria = X[:, split_col] <= split_val
        greater_criteria = X[:, split_col] > split_val
        if len(y[lesser_criteria]) < 3 or len(y[greater_criteria]) < 3:
            self.is_leaf = True
            self.y_vals = y
        else:
            self.l_child = dTree(self.k + 1).fit(X[lesser_criteria], y[lesser_criteria])
            self.r_child = dTree(self.k + 1).fit(X[greater_criteria], y[greater_criteria])
            
        return self

    def predict(self, X):
        if not self.l_child:
            return np.bincount(self.y_vals).argmax()
        if X[self.split_col] <= self.split_val:
            return self.l_child.predict(X)
        else:
            return self.r_child.predict(X)
    
    def gini(self, y):
        # get count of each unique value in the dataset
        _, counts = np.unique(y, return_counts=True)

        # gini impurity formula
        impurity = 1 - np.sum((counts / len(y))**2)
        return impurity


        

# note: all iris data is numeric

iris_df = sklearn.datasets.load_iris()
data = iris_df["data"]
target = iris_df["target"]

tree = dTree().fit(data, target)
print(tree.predict(iris_df["data"][a]))
print(iris_df["target"][a])

2
2


In [213]:
correct = 0
for i in range(len(iris_df["data"])):
    pred = tree.predict(iris_df["data"][i])
    true = iris_df["target"][i]
    if pred == true:
        correct += 1        
print(f"{correct / len(iris_df['data']) * 100}% accurate")

96.0% accurate
